In [1]:
import numpy as np
import gym
import tensorflow as tf 
import matplotlib.pyplot as plt

In [2]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Dense(32,input_dim=4,activation='relu'))
model.add(tf.keras.layers.Dense(2,activation='softmax'))
model.build()
optimizer=tf.keras.optimizers.Adam(lr=0.01)
compute_loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


In [3]:
def discount_rewards(r,gamma=0.8):
    discounted_r=np.zeros_like(r)
    running_add=0
    for t in reversed(range(0,r.size)):
        running_add=running_add*gamma+r[t]
        discounted_r[t]=running_add
    return discounted_r

In [ ]:
env=gym.make('CartPole-v0')
episodes=2000
scores=[]
update_every=5

gradBuffer=model.trainable_variables
for ix,grad in enumerate(gradBuffer):
    gradBuffer[ix]=grad*0
for e in range(episodes):
    state=env.reset()
    ep_memory=[]
    ep_score=0
    done=False
    while not done:
        state=state.reshape([1,4])
        env.render()
        with tf.GradientTape() as tape:
            logits=model(state)
            action_dist=logits.numpy()
            action=np.random.choice(action_dist[0],p=action_dist[0])
            action=np.argmax(action_dist==action)
            loss=compute_loss([action],logits)
        state,reward,done,_=env.step(action)
        ep_score+=reward
        if done:reward-=10
        grads=tape.gradient(loss,model.trainable_variables)
        ep_memory.append([grads,reward])
    scores.append(ep_score)
    
    ep_memory=np.array(ep_memory)
    ep_memory[:,1]=discount_rewards(ep_memory[:,1])
    
    for grads,reward in ep_memory:
        for ix,grad in enumerate(grads):
            gradBuffer[ix]+=grad*reward
    if e% update_every==0:
        optimizer.apply_gradients(zip(gradBuffer,model.trainable_variables))
        for ix,grad in enumerate(gradBuffer):
            gradBuffer[ix]=grad*0
    if e%100==0:
        print("Episode  {}  Score  {}".format(e, np.mean(scores[-100:])))

c:\users\박정은\gym\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Episode  0  Score  34.0
